In [74]:
import torch
from data.shapenet import ShapeNet
from model.vertix_model import VertixModel
import tqdm
from util.visualization import visualize_pointcloud, visualize_mesh
import random
%load_ext autoreload
%autoreload 2
import numpy as np
import pickle
from scipy.spatial import distance_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.utils import class_weight
from model.face_model import FaceModel
import torch_geometric.nn as nn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])
!pip install pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

Looking in links: https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/py37_cu113_pyt1102/download.html
     |████████████████████████████████| 30.0 MB 6.1 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 528 kB/s eta 0:00:01
     |████████████████████████████████| 596 kB 3.6 MB/s eta 0:00:01
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220119-py3-none-any.whl size=65267 sha256=408ab0341a57d10f14fd330dd80d642089a51a668321082fcd0595310a1a529c
  Stored in directory: /home/mohamed/.cache/pip/wheels/f3/b8/eb/61ed840f80d7198725bc061872b6019a7b3e9db4dbadf68083
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=b862c8b8702f43b0983e10cd4542ac1ca6d9ccd04d2a1634ea64db04affd4fbd
  Stored in directory: /home/mohamed/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built fvcore termcolor
ERROR: pip's dependency resolver does not currently take into account all the packages that are install

In [4]:
from pytorch3d.loss import chamfer_distance

ImportError: /home/mohamed/anaconda3/envs/pytorch11/lib/python3.7/site-packages/pytorch3d/_C.cpython-37m-x86_64-linux-gnu.so: undefined symbol: _ZNK2at6Tensor7is_cudaEv

In [75]:
num_vertices = 100

In [76]:
config = {
    'experiment_name': 'face_model_overfitting',
    'device': 'cuda:0',  
    'is_overfit': True,
    'batch_size': 8,
    'resume_ckpt': False,
    'learning_rate': 5e-4,
    'max_epochs': 50000,
    'print_every_n': 10,
    'validate_every_n': 50,
    'sdf_path': 'data/shapenet_dim32_sdf',
    'meshes_path': 'data/shapenet_reduced',
    'class_mapping': 'data/shape_info.json',
    'split': 'val',
    'num_vertices': num_vertices,
    'feature_size': 512,
    'num_trajectories': 2,
}

In [79]:
dataset = ShapeNet(sdf_path=config["sdf_path"],
                         meshes_path=config["meshes_path"],
                         class_mapping=config["class_mapping"],
                         split = "val", threshold=100, num_trajectories=config["num_trajectories"], face_mode=True)


In [80]:
dataset.filter_data()

Length of dataset: 32304
Filtering data ..


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32304/32304 [00:05<00:00, 5433.30it/s]

Length of dataset: 5474


In [43]:
from inference.inference_face_model import InferenceHandlerFaceModel

inferer = InferenceHandlerFaceModel('runs/face_model_chamfer/model_best.ckpt', config["num_vertices"], config["feature_size"], config["device"])

In [67]:
predicted_vertices, predicted_faces, adj, hv, gt_f = dataset[30]

In [68]:
visualize_mesh(predicted_vertices, predicted_faces)

Output()

In [69]:
refined_faces = inferer.infer_single(hv, adj.transpose(-1,0))

In [70]:
refined_faces.sum()

1220

In [71]:
predicted_faces.shape

(2596, 3)

In [72]:
output_faces = predicted_faces[refined_faces==1]

In [73]:
visualize_mesh(predicted_vertices, output_faces)

Output()